# Алгоритм Томпсона на батчах (без контекста)

1. На первом батче распределяем юзеров 50% на 50%.
2. Вероятность конверсии каждого варианта распределена по Beta распределению с $\alpha=1$,
$\beta=1$.
3. В конце каждого батча пересчитываем вероятности превосходства по точной формуле,
взятой отсюда https://www.johndcook.com//UTMDABTR-005-05.pdf
4. Распределяем трафик в пропорции вероятностей превосходства для каждого варианта
5. Останавливаем эксперимент при достижении определенной вероятности превосходства,
но не раньше определенного дня, чтобы учесть календарные факторы

Потенциальные проблемы:
- слишком рано отдаем трафик победителю
- из-за дисбаланса распределения трафика может быть больше успешных конверсий в этом варианте
(можно попробовать применить нормализацию)


***Реализуем алгоритм***

0. **Инициализация** - *BatchThompson(n_arms)*. Аргумент на вход: число вариантов сплита.
Здесь также инициализируются массивы для параметров Бета-распределений и вероятность превосходства = 0.5.

1. **Метод сплита** - *split_data()*. Исходя из вероятности превосходства вычисляем сплит по вариантам.
Возвращаем данные по конверсии на текущем батче для пересчета Бета-распределений.

2. **Метод изменения параметров распределения** - *.update_beta_params(data)*. Аргументы на вход: numpy массив со значениями конверсии по
каждому варианту. В случае неравномерного распределения по вариантам ставятся пропуски.
 - Проверяем, чтобы число столбцов совпадало с числом вариантов из инициализации.
 - Суммируем нули и единицы и обновляем параметры

3. **Метод пересчета** - *update_prob_super()*. Аргументов нет, так как учитывает измененные параметры
$\alpha$ (накопленное число успешных конверсий) и
$\beta$ (накопленное число неудачных конверсий) для всех вариантов.
 - Считаем по точной формуле
 - Выдаем массив из вероятностей превосходства

4. **Вероятность превосходства** - *prob_super_tuple()*. Аргументов нет, так как берем пересчитанные параметры.
5. **Критерий остановки** (*stopping_criterion*) - условия цикла while. Либо вероятность превосходства выше заданной
величины, либо закончились наблюдения.

In [2]:
from typing import List, Tuple
import os
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
from numpy import ndarray
from tqdm.notebook import tqdm
from src.ab import get_size_zratio
from src.mab import calc_prob_between, expected_loss,

# Графики
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")
from src.mab import BatchThompson, BatchThompsonMixed
from matplotlib.backends.backend_pdf import PdfPages
from joblib import Parallel, delayed

# Эксперименты на разных теоретических конверсиях и размерах батча (summation vs normalization)

Попробуем реализовать метод, описанный в статье
https://www.researchgate.net/publication/352117401_Parallelizing_Thompson_Sampling

Авторы предлагают следующий алгоритм. Пусть $k_a$ - число раз выбора руки $a$,
$l_a = 1$ - число раз выбора руки подряд.

Для каждого батча $t = 1, 2, .. T$:

- смотрим на вероятности бета распределений
- выбираем руку с наибольшей вероятностью
- присваиваем $k_a = k_a + 1$
- ЕСЛИ $k_a < 2^{l_a}$, то кидаем ВЕСЬ трафик в эту руку
- ИНАЧЕ: присваиваем $l_a = l_a + 1$ и распределяем трафик в ОБЕ руки ПОЛНОСТЬЮ (без долей)
- обновляем параметры и по новой

Утверждается, что он довольно хорошо работает и для динамических батчей - когда размер заранее нам неизвестен
Как выяснилось - работает не очень. Поэтому объединим два метода в один.

In [8]:
def thompson_results(algo_class, *index):
    p1_control, mde_test_effect, batch_size_share = index
    p_list = [p1_control, p1_control * (1 + mde_test_effect)]
    bts = algo_class(p_list_mu=p_list, batch_size_share_mu=batch_size_share)
    probability_superiority_steps, expected_loss_step_list, \
    observations_step_list, k_list_iter = bts.start_experiment()
    cumulative_observations_step_list = np.cumsum(observations_step_list, axis=0)
    print(p1_control, mde_test_effect, batch_size_share)
    return (probability_superiority_steps, expected_loss_step_list,
            cumulative_observations_step_list, bts.n_obs_every_arm, k_list_iter
            )

def plot_mab_results(directory, name, result_experiments_df):
    """
    Function creates two lines in one plot: left axis - probability superiority, right - cummulative observation
    :param p_list_mu: expectation for conversion rates
    :param batch_size_share_mu: expectation for batch size
    :param probability_superiority_steps:
    :param cumulative_observations_step_list:
    :param folder:
    :return:
    """
    plot_file = PdfPages(f"{directory}/{name}.pdf")
    for index, row in result_experiments_df.iterrows():
        p1_control, mde_test_effect, batch_size_share = index
        plt.figure(figsize=(15, 7));
        x = np.arange(1, row['probability_superiority_steps'].shape[0]+1)
        fig, ax1 = plt.subplots();

        ax2 = ax1.twinx();
        ax1.plot(x, row['cumulative_observations_step_list'][:, 0], '--', color='b', label='data1');
        ax1.plot(x, row['cumulative_observations_step_list'][:, 1], '--', color='r', label='data2');
        ax2.plot(x, np.array(row['probability_superiority_steps'])[:, 1], '-', color='r');
        ax2.plot(x, row['expected_loss_steps'], ':', color='r')
        plt.title(f"p_list_mu: {[p1_control, p1_control * (1 + mde_test_effect)]} \n"
                  f"batch_size_share_mu, %: {batch_size_share * 100} \n "
                  f"dash lines - observations; red line - probability to win for 2 variant",
                  fontdict={"size": 5});
        plot_file.savefig();
        plt.close();
    plot_file.close();
    return



p1_control = np.round(np.linspace(0.01, 0.5, 30), 3)
mde_test_effect = np.round(np.linspace(0.01, 0.15, 2), 3)
batch_size_share = np.round(np.linspace(0.001, 0.1, 10), 3)

result_experiments_df = pd.DataFrame(index=pd.MultiIndex.from_product(
                                     [p1_control, mde_test_effect, batch_size_share],
                                     names=["p1", "mde", "batch_size_share"]),
                                     columns=['probability_superiority_steps',
                                              'expected_loss_steps',
                                              'cumulative_observations_step_list',
                                              'n_obs_per_every_arm',
                                              'k_list_iter'])

algo_class = BatchThompsonMixed
results_all =  Parallel(n_jobs=-1)(
                             delayed(thompson_results)(algo_class, *index)
                             for index, row in tqdm(result_experiments_df.iterrows())
                             )
i = 0
for index, row in  result_experiments_df.iterrows():
    result_experiments_df.loc[index, "probability_superiority_steps"] = results_all[i][0]
    result_experiments_df.loc[index, "expected_loss_steps"] = results_all[i][1]
    result_experiments_df.loc[index, "cumulative_observations_step_list"] = results_all[i][2]
    result_experiments_df.loc[index, "n_obs_per_every_arm"] = results_all[i][3]
    result_experiments_df.loc[index, "k_list_iter"] = results_all[i][4]
    i += 1
plot_mab_results("Plot/Thompson/Experiment4", "ThompsonMixed_new",
                 result_experiments_df)

0it [00:00, ?it/s]

0.01 0.15 0.012
0.044 0.15 0.045
0.044 0.15 0.056
0.061 0.15 0.045
0.061 0.15 0.056
0.078 0.15 0.001
0.078 0.15 0.012
0.145 0.15 0.023
0.162 0.01 0.034
0.162 0.01 0.1
0.162 0.15 0.012
0.179 0.01 0.1
0.179 0.15 0.045
0.179 0.15 0.067
0.179 0.15 0.089
0.196 0.01 0.023
0.196 0.15 0.045
0.196 0.15 0.078
0.196 0.15 0.1
0.213 0.01 0.045
0.213 0.01 0.078
0.213 0.15 0.023
0.213 0.15 0.056
0.213 0.15 0.078
0.213 0.15 0.1
0.23 0.01 0.012
0.23 0.01 0.089
0.23 0.15 0.001
0.331 0.01 0.034
0.331 0.01 0.045
0.331 0.01 0.078
0.331 0.15 0.001
0.416 0.15 0.045
0.416 0.15 0.067
0.416 0.15 0.1
0.432 0.01 0.023
0.432 0.01 0.034
0.432 0.01 0.045
0.432 0.01 0.078
0.432 0.15 0.001
0.5 0.01 0.067
0.5 0.15 0.012
0.01 0.15 0.001
0.128 0.15 0.012
0.128 0.15 0.023
0.128 0.15 0.034
0.128 0.15 0.045
0.128 0.15 0.056
0.128 0.15 0.089
0.145 0.01 0.001
0.263 0.01 0.045
0.263 0.01 0.067
0.263 0.01 0.1
0.263 0.15 0.012
0.28 0.01 0.012
0.28 0.01 0.1
0.28 0.15 0.034
0.28 0.15 0.045
0.28 0.15 0.067
0.28 0.15 0.078
0.297 0.0

KeyboardInterrupt: 

In [10]:
get_size_zratio(0.5, 0.5 * (1 + 0.01), alpha=0.05, beta=0.2)





25897